# 4.3 Accessing metadata via Python from a Jupyter Notebook
Now that we know how searching for metadata works through geoportals, we want to look a bit more behind the HTML interface. When you're filtering the datasets there are requests executed on the CSW in the background. The construction of the requests is always the same at CSWs. But before we can start let's quickly set up our all other components which we need. The OGC have an own Python library, called OWSLib, for client programming. This library allows the users to access and utilize geospatial data from their online services, like WMS or CSW, via Python. After installing the library you will also need to import the CSW class from OWSLib and some methods for later use. 

In [1]:
%pip install OWSLib
from owslib.csw import CatalogueServiceWeb
from owslib.fes import PropertyIsEqualTo, BBox, PropertyIsLike


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: C:\Users\Tobias\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Now we need the URL from the INSPIRE CSW. I've you search for the "INSPIRE CSW" in your browser, you should get the the catalogue service in one of the first links (https://inspire-geoportal.ec.europa.eu/GeoportalProxyWebServices/resources/OGCCSW202?request=GetCapabilities&service=CSW). It's the GetCapabilities request from the CSW. In the GetCapabilities you get all the metadata for the catalogue service.
But to use the catalogue service from INSPIRE in the CSW class from OWSLib you only need the first part of the URL until the "?". With that and the function CatalogueServiceWeb(), we can create a connection to the CSW. 

In [26]:
CSW_URL = 'https://inspire-geoportal.ec.europa.eu/GeoportalProxyWebServices/resources/OGCCSW202'
csw = CatalogueServiceWeb(CSW_URL)

Let's first see which operations the INSPIRE CSW has.

In [ ]:
[op.name for op in csw.operations]

So we need the metadata about the GetRecords operation before we can search for a dataset. We want to know for what values we can search for.

In [ ]:
for constraint in csw.constraints:
    print(f"  {constraint}")


Now look at the SupportedISOQueryables and search for the values of the contraint. They are listed two times in GetRecords but we only want them listed one time to get an better oerview.

In [ ]:
temp = 0
for constraint in op.constraints:
    if 'SupportedISOQueryables' in constraint.name and temp == 0:
        temp += 1
        print(constraint.values)

There are quite a few values which we can use. But not all are very helpful for us.

In [29]:
hannover_query = PropertyIsLike('csw:title', '%Hannover%')
csw.getrecords2(constraints=[hannover_query])
csw.results

{'matches': 35, 'returned': 10, 'nextrecord': 11}

To output your for example the titles of your results you can make a for loop and print the records.

In [31]:
for rec in csw.records:
    print(csw.records[rec].title)

You can also try to filter with BBox() and that will make it possible filter for datasets in a specific area.

In [27]:
bbox_query = BBox([52.244094, 9.212036, 52.574420, 10.318909])
csw.getrecords2(constraints=[bbox_query])
csw.results

{'matches': 385, 'returned': 10, 'nextrecord': 11}

If you know the identifier of our wanted dataset you can also search for that.

In [28]:
#print(csw.getrecords2(constraints=[PropertyIsEqualTo('dc:identifier', '412966c0-a49f-40d2-8419-d7eca338da0f')], maxrecords=1))
csw.getrecordbyid(id=['412966c0-a49f-40d2-8419-d7eca338da0f'])
csw.records['412966c0-a49f-40d2-8419-d7eca338da0f'].title


KeyError: '412966c0-a49f-40d2-8419-d7eca338da0f'

In [ ]:
# Suchkriterien definieren
suchtext = "Hannover"
suchattribut = "Title"

# Filter erstellen
filter = PropertyIsLike(propertyname=suchattribut, literal=f"*{suchtext}*", escapeChar="\\", wildCard="*")

# GetRecords-Anfrage mit Filter durchführen
response = csw.getrecords2(constraints=[filter], maxrecords=10)

# Überprüfen, ob Ergebnisse vorhanden sind
if response is not None:
    # Ergebnisse auswerten
    for rec in response.records:
        print(rec.title)
else:
    print("Keine Ergebnisse gefunden.")

In [ ]:
rec = csw.records[0]
print(rec.title)
print(rec.references)

In [ ]:
caps = csw.getcapabilities()

# verfügbare Suchkriterien ausgeben
for op in csw.operations:
    if 'GetRecords' in op.name:
        print(f"\nSearch criteria for {op.name}:")
        for constraint in op.constraints:
            print(f"  {constraint.name}")

In [ ]:
csw.getrecords2()
print(csw.results)

In [3]:
hannover_query = PropertyIsLike('csw:title', '%Hannover%')
csw.getrecords2(constraints=[hannover_query])
csw.results

{'matches': 35, 'returned': 10, 'nextrecord': 11}

In [4]:
for recid in csw.records:
    record = csw.records[recid]
    print(record.title)

-------
Test documantation   

csw2 = CatalogueServiceWeb('https://gdk.gdi-de.org/gdi-de/srv/ger/csw')   
    
hannover_query = PropertyIsLike('csw:title', '%Hannover%')   
csw2.getrecords2(constraints=[hannover_query])   
csw2.results   

for rec in csw2.records:
    print(csw2.records[rec].title)

In [23]:
#Test documentation
csw2 = CatalogueServiceWeb('https://gdk.gdi-de.org/gdi-de/srv/ger/csw')

hannover_query = PropertyIsLike('csw:title', '%Hannover%')
csw2.getrecords2(constraints=[hannover_query])
csw2.results


{'matches': 83, 'returned': 10, 'nextrecord': 11}

In [22]:

for rec in csw2.records:
    print(csw2.records[rec].title)

Naturschutzgebiete in der Region Hannover
INSPIRE Downloaddienst (WFS) - Naturschutzgebiete - originär (Region Hannover)
INSPIRE Darstellungsdienst (WMS) - Naturschutzgebiete - originär (Region Hannover)
INSPIRE Downloaddienst (ATOM-Feed) - Naturschutzgebiete - originär (Region Hannover)


In [20]:
# wanted dataset
csw2.getrecordbyid(id=['412966c0-a49f-40d2-8419-d7eca338da0f'])
csw2.records['412966c0-a49f-40d2-8419-d7eca338da0f'].title

'Naturschutzgebiete in der Region Hannover'